<a href="https://colab.research.google.com/github/Mohan0332/Capstone/blob/main/Feature%20extract%20and%20TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
#for resnet50 model

In [ ]:
import os
import csv
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.svm import LinearSVC

# Define the directory containing the train and test datasets
train_dir = '/content/drive/MyDrive/Capstone dataset/Images-out_RGB/train'
test_dir = '/content/drive/MyDrive/Capstone dataset/Images-out_RGB/test'

# Load the VGG16 model with pre-trained weights and exclude the top layers
res = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Define a function to extract features from an image using the VGG16 model
def extract_features(img_path):
    # Load the image and preprocess it for the VGG16 model
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Use the VGG16 model to extract features
    features = res.predict(x)

    # Flatten the features to a 1-dimensional array
    features = features.flatten()

    return features

# Define a function to extract features from all images in a directory
def extract_directory_features(directory):
    features = []
    labels = []
    for root, dirs, files in os.walk(directory):
        for filename in files:
            # Check if the file is an image
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                # Get the full path of the image file
                img_path = os.path.join(root, filename)

                # Extract the features from the image using VGG16
                img_features = extract_features(img_path)

                # Add the features and label to the lists
                features.append(img_features)
                labels.append(int(root.split('/')[-1]))

    return np.array(features), np.array(labels)

# Extract features and labels from the train and test datasets
train_features, train_labels = extract_directory_features(train_dir)
test_features, test_labels = extract_directory_features(test_dir)

In [16]:
train_features.shape

(490, 100352)

In [17]:
# Fit a linear SVM on the train features and labels
svm = LinearSVC()
svm.fit(train_features, train_labels)

LinearSVC()

In [18]:
# Evaluate the SVM on the test features and labels
accuracy = svm.score(test_features, test_labels)
print('Test accuracy:', accuracy)

Test accuracy: 0.6491228070175439


In [19]:
from sklearn.linear_model import LogisticRegression

Logreg = LogisticRegression()
Logreg.fit(train_features, train_labels)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [20]:
accuracy = Logreg.score(test_features,test_labels)

In [21]:
accuracy

0.6666666666666666

In [26]:
#MobilenetV2

In [25]:
#for MobilenetV2 model
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Define the directories containing the training and testing data
train_dir = '/content/drive/MyDrive/Capstone dataset/Images-out_RGB/train'
test_dir = '/content/drive/MyDrive/Capstone dataset/Images-out_RGB/test'

# Define the MobileNetV2 model
mobile_net = MobileNetV2(include_top=False, pooling='avg')

# Define a function to extract features from a given image file
def extract_features(file_path):
    img = image.load_img(file_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x = np.expand_dims(x, axis=0)
    features = mobile_net.predict(x)
    return features

# Define a function to extract features from a directory of images
def extract_features_from_dir(dir_path):
    features = []
    labels = []
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if file.endswith(('.jpg','.png')):
                file_path = os.path.join(root, file)
                label = os.path.basename(root)
                feature = extract_features(file_path)
                features.append(feature)
                labels.append(label)
    return np.array(features), np.array(labels)

# Extract features from the training and testing directories
train_features, train_labels = extract_features_from_dir(train_dir)
test_features, test_labels = extract_features_from_dir(test_dir)

# Flatten the features for use in a linear SVM
train_features_flat = train_features.reshape(train_features.shape[0], -1)
test_features_flat = test_features.reshape(test_features.shape[0], -1)

1/1 [==============================] - 0s 41ms/step


In [27]:
# Convert the labels to integers
train_labels = np.array([int(label) for label in train_labels])
test_labels = np.array([int(label) for label in test_labels])

# Train a linear SVM on the flattened features
from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(train_features_flat, train_labels)

# Evaluate the SVM on the test data
accuracy = svm.score(test_features_flat, test_labels)
print('Accuracy:', accuracy)

Accuracy: 0.7719298245614035


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [29]:
# Train a linear SVM on the flattened features
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(train_features_flat, train_labels)

# Evaluate the SVM on the test data
accuracy = logreg.score(test_features_flat, test_labels)
print('Accuracy:', accuracy)

Accuracy: 0.7894736842105263


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#